# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [29]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in these examples:"
link_system_prompt += """
`{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}`
"""
link_system_prompt += """
`{
    "links": [
        {"type": "products", "url": "https://website.com/products"},
        {"type": "social media": "url": "https://facebook.com/organization/page"}
    ]
}`
"""

In [30]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in these examples:
`{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}`

`{
    "links": [
        {"type": "products", "url": "https://website.com/products"},
        {"type": "social media": "url": "https://facebook.com/organization/page"}
    ]
}`



In [31]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [32]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [33]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [34]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Qwen/Qwen-Image-Edit',
 '/deepseek-ai/DeepSeek-V3.1-Base',
 '/xai-org/grok-2',
 '/deepseek-ai/DeepSeek-V3.1',
 '/ByteDance-Seed/Seed-OSS-36B-Instruct',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Qwen/Qwen-Image-Edit',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/lvwerra/jupyter-agent-2',
 '/spaces/NXN-Labs/Voost',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/nvidia/Granary',
 '/datasets/nvidia/Llama-Nemotron-VLM-Dataset-v1',
 '/datasets/nvidia/Nemotron-CC-v2',
 '/datasets/nvidia/Nemotron-Post-Training-Dataset-v2',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/

In [35]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [36]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [37]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'models', 'url': 'https://huggingface.co/models'}, {'type': 'datasets', 'url': 'https://huggingface.co/datasets'}, {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine l

In [38]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [39]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [40]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'models', 'url': 'https://huggingface.co/models'}, {'type': 'datasets', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nQwen/Qwen-Image-Edit\nUpdated\nabout 21 hours ago\n•\n40.4k\n•\n1.33k\ndeepseek-ai/DeepSeek-V3.1-Base\nUpdated\n4 days ago\n•\n15.3k\n•\n921\nxai-org/grok-2\nUpdated\n2 days ago\n•\n1.39k\n•\n687\ndeepseek-ai/DeepSeek-V3.1\nUpdated\n4 days ago\n•\n21.4k\n•\n556\nByteDance-Seed/Seed-OSS-36B-Instruct\nUpdated\nabout 14 hours ago\n•\n6.36k\n•\n323\nBrowse 1M+ models\nSpaces\nRunning\n12.3k\n1

In [41]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [42]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'models', 'url': 'https://huggingface.co/models'}, {'type': 'datasets', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Welcome to Hugging Face
**The AI community building the future.**  
At Hugging Face, we are at the forefront of machine learning, providing a collaborative platform where developers, researchers, and organizations can explore and share AI models, datasets, and applications. With over a million models and extensive community engagement, we’re committed to making AI accessible, efficient, and transformative.

---

## Company Culture
At Hugging Face, we prioritize **collaboration, innovation, and community**. Our team embraces a diverse culture that values openness and continuous learning. We foster an environment where all ideas are welcomed, allowing individuals to thrive and contribute to groundbreaking projects in AI. Whether you are a novice or an expert, your contributions to our community are invaluable.

---

## Our Customers
Hugging Face serves a broad spectrum of users, from independent developers to large enterprises. Notable organizations leveraging our platform include:

- **Meta** (AI at Meta)
- **Amazon**
- **Google**
- **Microsoft**
- **Grammarly**

We empower over **50,000 organizations** to accelerate their AI development through our state-of-the-art infrastructure and open-source tools.

---

## Products and Services
### Models & Datasets
- **Browse 1M+ Models:** Access a vast collection of pre-trained AI models across diverse domains such as text processing, image recognition, and audio analysis.
- **Datasets:** Utilize over **250k datasets** for training and evaluation, enabling comprehensive research and development.
  
### Spaces
Join a vibrant community of users building and sharing applications. Our platform hosts **400k+ applications**, allowing for seamless collaboration and innovation.

---

## Careers at Hugging Face
We are always on the lookout for passionate individuals to join our team. Our roles span various domains, including AI research, software development, data science, and customer support. At Hugging Face, you can expect:

- **Innovation-centered Projects:** Work on cutting-edge AI technologies.
- **Collaborative Culture:** Engage with a community that values diverse perspectives and ideas.
- **Professional Growth:** Opportunities to enhance your skills and advance your career.

If you're ready to make an impact in the AI space, we encourage you to visit our [Jobs Page](#) for current openings.

---

## Join Us
Explore the future of AI with us! Become a part of our dynamic community today—**[Sign Up](#)** and start collaborating, building, and sharing.

---

For more information about our services, products, and community initiatives, visit us at: [Hugging Face](https://huggingface.co)

---

*Together, let's build a smarter future with AI!*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [47]:
def process_model_response(stream):
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
    return response

def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    return process_model_response(stream)

def create_spanish_brochure(brochure_markdown):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Translate the following content into Spanish, keeping the Markdown formatting."},
            {"role": "user", "content": brochure_markdown}
          ],
        stream=True
    )
    return process_model_response(stream)


In [44]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'github', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Hugging Face: Building the Future of AI

**Welcome to Hugging Face**  
The AI community that empowers collaboration and innovation in machine learning. 

## Our Mission
At Hugging Face, our vision is to foster an open and inclusive environment where the machine learning community can collaborate on models, datasets, and applications. We believe that by coming together, we can build better and more advanced AI technologies.

## What We Offer

### Collaboration Platform
- **Models:** Access and collaborate on over 1 million cutting-edge AI models.
- **Datasets:** Share and utilize a vast library of over 250,000 datasets for any ML task.
- **Spaces:** Create and host applications that leverage AI capabilities.

### For Businesses
Our Enterprise solutions provide secure, scalable, and robust AI infrastructure tailored for organizations of all sizes. Key features include:
- Advanced security frameworks
- Dedicated support
- Team collaboration tools

### Community Engagement
Join more than 50,000 organizations—including industry leaders like Google, Microsoft, and Amazon—who trust us for their AI needs.

## Company Culture
Hugging Face is dedicated to creating a caring and inclusive company culture that prioritizes collaboration, learning, and shared growth. We are passionate about open-source development and actively seek to engage our community in building the future of ML tools. 

### Our Values
- **Community:** We believe in the power of collaboration and the value of diverse perspectives.
- **Transparency:** We are committed to open practices, sharing knowledge, and fostering trust within our community.
- **Innovation:** We encourage creative thinking and problem-solving to advance technological frontiers. 

## Careers at Hugging Face
We are always on the lookout for talented, passionate individuals who aspire to make a difference in the AI space. Join us to work on cutting-edge technologies, be part of a dynamic team, and contribute to meaningful projects that have a global impact.

- **Explore Current Openings:** Check out our [Jobs Page](https://huggingface.co/jobs) and discover your next opportunity to grow your career with us!

## Join the AI Revolution
Explore our platform, collaborate with innovators, and contribute to the evolution of machine learning. Whether you are a developer, researcher, or business leader, Hugging Face inspires everyone to take part in shaping the future of AI.

**Contact Us**  
- Email: contact@huggingface.co  
- Follow us: [Twitter](https://twitter.com/huggingface), [LinkedIn](https://linkedin.com/company/huggingface), [Discord](https://discord.gg/huggingface) 

Join us today!



In [48]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

system_prompt = "You are a snarky assistant that analyzes the contents of several relevant pages from a company website \
and creates a short sarcastic brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

brochure = stream_brochure("HuggingFace", "https://huggingface.co")
print("-"*50)
spanish_brochure = create_spanish_brochure(brochure)

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Welcome to Hugging Face: Your AI Playground 🥳

## Who are we? 

Oh, just your neighborhood **AI community** that claims to be "building the future." We're basically the hipsters of machine learning—collaborating on **models, datasets, and applications** like it's our full-time job. Spoiler alert: it is!

## What’s in the Box? 🎁 

- **1M+ Models**: Yes, you heard that right. Why use Wordle when you can play around with over a million AI models instead? Indecision is now infinite! 
- **250K+ Datasets**: Drowning in data? We offer so much variety, you’ll wish you never asked where all this info is coming from.
- **AI Apps and Spaces**: Hosting applications so cool, they practically make you want to quit your full-time job and start coding in your mom's basement. 

## Our Customers 💼

We have **more than 50,000 organizations** swimming in our love pool, including tech giants like **Meta, Amazon, Google, and Microsoft**. Clearly, they see the value... because they definitely don’t have enough data already.

## Culture: Be a Part of Our ‘Code’-pendent Relationship 💻❤️

Want to work with the best? Join our team of master coders and future-seers! Come bask in our thrilling environment where work-life balance is just a fancy phrase. We have:

- **Snap-worthy Office Vibes**: Open spaces and possibly too much coffee.
- **(Un)limited Collaboration**: Because if you’re not collaborating, are you even trying?
- **Profit-Driven Enthusiasm**: We care about your growth, but let's be real, we love it when you stick around for more than just the free snacks.

## Careers: Jump on this Rollercoaster 🎢

Looking for a job? Well, we've got some! Sadly, you won't just be "hugging faces"—you'll be crunching code and training models. Join us if you're cool with working hard, playing hard, and using **shiny** tools like Transformers and Diffusion models.

### Benefits Include:
- **Competitive Pay**: Because we totally understand that you’d like to pay your rent.
- **Single Sign-On**: Isn’t the tech world charming? You’ll have access to everything with just one click! 
- **Priority Support**: If you break it, don’t worry! We promise to prioritize your cries for help (depending on how busy we are).

## In a Nutshell:

Hugging Face is here to save the day, and we want you to join this grand adventure into the world of AI. Or just enjoy watching from the sidelines as we build the tech future you always dreamed of! 

So, if you want to rub elbows with the elite of machine learning—and think hugging virtual faces is as cool as it sounds—**sign up today**! 🎉

--------------------------------------------------


# Bienvenido a Hugging Face: Tu Parque de Diversiones de IA 🥳

## ¿Quiénes somos?

Oh, solo tu **comunidad de IA** del vecindario que afirma estar "construyendo el futuro". Básicamente, somos los hipsters del aprendizaje automático—colaborando en **modelos, conjuntos de datos y aplicaciones** como si fuera nuestro trabajo a tiempo completo. ¡Spoiler: lo es!

## ¿Qué hay en la Caja? 🎁 

- **Más de 1M de Modelos**: Sí, lo oíste bien. ¿Por qué usar Wordle cuando puedes jugar con más de un millón de modelos de IA? ¡La indecisión ahora es infinita! 
- **Más de 250K Conjuntos de Datos**: ¿Te ahogas en datos? Ofrecemos tanta variedad, que desearás no haber preguntado de dónde viene toda esta información.
- **Aplicaciones y Espacios de IA**: Hospedamos aplicaciones tan geniales, que prácticamente te harán querer renunciar a tu trabajo a tiempo completo y empezar a programar en el sótano de tu mamá. 

## Nuestros Clientes 💼

Tenemos **más de 50,000 organizaciones** nadando en nuestro mar de amor, incluyendo gigantes tecnológicos como **Meta, Amazon, Google y Microsoft**. Claramente, ven el valor... porque definitivamente no tienen suficientes datos ya.

## Cultura: Sé Parte de Nuestra Relación 'Code'-pendiente 💻❤️

¿Quieres trabajar con los mejores? ¡Únete a nuestro equipo de maestros codificadores y videntes del futuro! Ven a disfrutar de nuestro emocionante entorno donde el equilibrio entre la vida laboral y personal es solo una frase elegante. Tenemos:

- **Vibras de Oficina para Instagram**: Espacios abiertos y posiblemente demasiado café.
- **Colaboración (In)finita**: Porque si no estás colaborando, ¿realmente lo estás intentando?
- **Entusiasmo Impulsado por el Beneficio**: Nos importa tu crecimiento, pero seamos sinceros, nos encanta cuando te quedas más allá de los bocadillos gratis.

## Carreras: Súbete a esta Montaña Rusa 🎢

¿Buscando trabajo? ¡Bueno, tenemos algunos! Lamentablemente, no solo estarás "abrazando caras"—estarás procesando código y entrenando modelos. Únete a nosotros si te parece bien trabajar duro, jugar duro, y usar herramientas **brillantes** como Transformadores y modelos de Difusión.

### Beneficios Incluyen:
- **Salario Competitivo**: Porque entendemos perfectamente que te gustaría pagar tu alquiler.
- **Inicio de Sesión Único**: ¿No es encantador el mundo tecnológico? ¡Tendrás acceso a todo con solo un clic! 
- **Soporte Prioritario**: Si lo rompes, ¡no te preocupes! Prometemos priorizar tus gritos de ayuda (dependiendo de cuán ocupados estemos).

## En Resumen:

Hugging Face está aquí para salvar el día, y queremos que te unas a esta gran aventura en el mundo de la IA. O simplemente disfrutar mirando desde las gradas mientras construimos el futuro tecnológico con el que siempre soñaste. 

Así que, si quieres codearte con la élite del aprendizaje automático—y piensas que abrazar caras virtuales es tan genial como suena—**¡inscríbete hoy!** 🎉

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>